In [1]:
# Необходимые библиотеки
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
import re
import pandas as pd

In [2]:
# Сохранение путей и параметров в переменные
url = 'https://hh.ru'
params = {'clusters': 'true',
          'ored_clusters': 'true',
          'enable_snippets': 'true',
          'salary': '',
          'text': 'data scientist',
          'page': 0}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'}

In [3]:
vacancies_list = []
while True:
    response = requests.get(url + '/search/vacancy/', params=params, headers=headers)
    dom = bs(response.text, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item'})
    
    if response.ok and vacancies:
        for vacancy in vacancies:
            vacancies_data = {}
            name = vacancy.find('a', {'class': 'bloko-link'}).text
            link = vacancy.find('a', {'class': 'bloko-link'})['href']
            company_name = vacancy.find('div', {'class': 'vacancy-serp-item__meta-info-company'}).text
            adress = vacancy.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).text
            wage = []
            try:
                for i in re.split(" – | ", vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).text):
                    i = i.split('\u202f')
                    wage.append(''.join(i))
                min_wage = int(wage[0])
                max_wage = int(wage[1])
                currency = wage[2]
            except:
                try:
                    if wage[0] == 'до':
                        min_wage = None
                        max_wage = int(wage[1])
                        currency = wage[2]
                    elif wage[0] == 'от':
                        min_wage = int(wage[1])
                        max_wage = None
                        currency = wage[2]
                except:
                    min_wage = None
                    max_wage = None
                    currency = None
            vacancies_data['name'] = name
            vacancies_data['link'] = link
            vacancies_data['company_name'] = company_name
            vacancies_data['adress'] = adress
            vacancies_data['min_wage'] = min_wage
            vacancies_data['max_wage'] = max_wage
            vacancies_data['currency'] = currency
            vacancies_list.append(vacancies_data)
        params['page'] += 1
    else:
        break

In [4]:
df = pd.DataFrame.from_dict(vacancies_list)
df.to_csv(f'hh_{params["text"]}.csv')

In [5]:
df

,name,link,company_name,adress,min_wage,max_wage,currency
0,Senior Data Scientist,https://hhcdn.ru/click?b=253624&place=35,Сбер. Data Science,Сочи,NaN,NaN,None
1,"Team Lead Data Scientist (RecSys, NLP, CV)",https://hhcdn.ru/click?b=253623&place=36,Сбер. IT,Москва,NaN,NaN,None
2,Data Scientist (команда Поиска),https://hh.ru/vacancy/46048820?from=vacancy_se...,HeadHunter::Analytics/Data Science,"Москва, Алексеевская",NaN,NaN,None
3,Python Developer (Data Science),https://hh.ru/vacancy/49232800?from=vacancy_se...,ООО IHS Global,Москва,NaN,NaN,None
4,Data - инженер,https://hh.ru/vacancy/48927705?from=vacancy_se...,Сбер. Data Science,Москва,NaN,NaN,None
...,...,...,...,...,...,...,...
315,Product Marketing Manager (DataSpell),https://hh.ru/vacancy/48576039?from=vacancy_se...,JetBrains,Москва,150000.0,NaN,руб.
316,QA - инженер (нагрузочное тестирование),https://hh.ru/vacancy/47775277?from=vacancy_se...,Сбер. IT,Москва,NaN,NaN,None
317,Python-разработчик,https://hh.ru/vacancy/48493498?from=vacancy_se...,Сбер. IT,"Москва, Кутузовская",NaN,NaN,None
318,Тестировщик (QA engineer),https://hh.ru/vacancy/48376438?from=vacancy_se...,ООО РАБЛЗ,Москва,120000.0,180000.0,руб.
